In [2]:
import os                         # 파일과 디렉토리 경로 관리
import kagglehub                  # 캐글 데이터셋 불러옴
import numpy as np                # 배열
import pandas as pd               # 데이터 조작 및 분석용 데이터프레임
from scipy.io import wavfile      # wav 파일 처리
from sklearn.model_selection import train_test_split      # 데이터셋을 학습용과 테스트용으로 분리
from sklearn.ensemble import RandomForestClassifier       # 랜덤포레스트 알고리즘
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # 모델성능평가도구
import seaborn as sns             # 데이터 시각화 라이브러리
import matplotlib.pyplot as plt   # 그래프 시각화

In [3]:
# KaggleHub을 이용해 데이터 다운로드
path = kagglehub.dataset_download("andradaolteanu/gtzan-dataset-music-genre-classification")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\tlswn\.cache\kagglehub\datasets\andradaolteanu\gtzan-dataset-music-genre-classification\versions\1


In [4]:
# 경로 설정
audio_folder = os.path.join(path, "Data/genres_original")
genres = os.listdir(audio_folder)

In [5]:
data_path=os.path.join(path,"Data/features_3_sec.csv")
data = pd.read_csv(data_path)

# 데이터의 첫 5줄 확인
print(data.head())

# filename - 오디오 파일 이름 (장르.인덱스.세그먼트번호)
# length - 오디오 파일 길이 
# chroma_stft_mean(평균) & chroma_stft_var(분산) (Chroma Short-Time Flourier Transform) 
# - 주파수 스펙트럼 크기 12개 피치 클래스로 요악, 멜로디 또는 화음과 관련된 정보 캡처
# rms_mean & rms_var (Root Mean Square) - 신호 에너지 측정(오디오 음량크기)
# spectral_centroid_mean & spectral_centroid_var 
# - 스펙트럼 중심 (가중 평균 주파수), 음색 특징 짓는데 사용
# spectral_bandwidth_mean & spectral_bandwidth_var 
# - 스펙트럼 대역폭 (주파수 분포 폭), 음색과 다이나믹 구분에 도움
# rolloff_mean & rolloff_var - Spectral Rolloff 신호의 총 에너지 중 특정 백분율(약 85%)이 포함되는 주파수,
# - 오디오의 고주파수 콘텐츠
# zero_crossing_rate_mean & zero_crossing_rate_var 
# - 신호값이 0을 지나칠 때의 빈도(진동성), 소리의 음질과 텍스처 분석
# harmony_mean & harmony_var - 신호의 하모니 (화음)
# perceptr_mean & perceptr_var - 오디오의 지각적 특징(음향적 감각 관련), 사람의 청각적 특성을 모델링.
# tempo - 음악의 속도(분당 비트 수), 음악의 리듬 표현
# mfcc1_mean ~ mfcc20_var - MFCC (Mel-Frequency Cepstral Coefficients) 주파수 도메인에서 음향 특징을 요약한 값, 음성 신호 처리와 음악 분류에서 가장 중요한 특징
# label - 오디오 장르

            filename  length  chroma_stft_mean  chroma_stft_var  rms_mean  \
0  blues.00000.0.wav   66149          0.335406         0.091048  0.130405   
1  blues.00000.1.wav   66149          0.343065         0.086147  0.112699   
2  blues.00000.2.wav   66149          0.346815         0.092243  0.132003   
3  blues.00000.3.wav   66149          0.363639         0.086856  0.132565   
4  blues.00000.4.wav   66149          0.335579         0.088129  0.143289   

    rms_var  spectral_centroid_mean  spectral_centroid_var  \
0  0.003521             1773.065032          167541.630869   
1  0.001450             1816.693777           90525.690866   
2  0.004620             1788.539719          111407.437613   
3  0.002448             1655.289045          111952.284517   
4  0.001701             1630.656199           79667.267654   

   spectral_bandwidth_mean  spectral_bandwidth_var  ...  mfcc16_var  \
0              1972.744388           117335.771563  ...   39.687145   
1              2010.05

In [6]:
# 열 이름 확인
print("Column names:", data.columns)

Column names: Index(['filename', 'length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean',
       'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'm

In [7]:
# 특성과 레이블 저장
features = []
labels = []

In [8]:
# 특성 추출 함수 (scipy 활용)
def extract_features_with_scipy(file_path, genre_label):
    sr, audio = wavfile.read(file_path)  # 오디오 파일 로드
    rms = np.sqrt(np.mean(audio ** 2))  # RMS 에너지
    mean = np.mean(audio)  # 평균
    std = np.std(audio)  # 표준편차
    max_val = np.max(audio)  # 최대값
     # 특성 저장필요
    labels.append(genre_label)  # 레이블 저장